In [ ]:
import io, os, sys
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

class LazySegmentTree:
  def __init__(self, data):
    self.size = 1 << (len(data) - 1).bit_length()
    self.L = [0] * (2 * self.size)
    self.lazy = [0] * (2 * self.size)
    self.L[self.size:self.size + len(data)] = data
    for i in reversed(range(self.size)):
      self.L[i] = self.L[i*2] + self.L[i*2+1]

  def push(self, i, s, e) :
    if not self.lazy[i] : return

    if i < self.size :
      self.lazy[i*2] += self.lazy[i]
      self.lazy[i*2+1] += self.lazy[i]
    elif self.lazy[i] & 1: #리프노드이면서 lazy[i]가 홀수면
      self.L[i] ^= 1 #반전
    self.lazy[i] = 0

  def query(self, l, r):
    return self._query(l, r, 1, 0, self.size)

  def _query(self, l, r, i, nl, nr) :
    self.push(i, nl, nr)
    
    if r <= nl or nr <= l : return 0
    if l <= nl and nr <= r : return self.L[i]
    mid = (nl + nr) // 2
    return self._query(l, r, i*2, nl, mid) + self._query(l, r, i*2+1, mid, nr)
  
  def add(self, l, r, x):
    self._add(l, r, x, 1, 0, self.size)

  def _add(self, l, r, x, i, nl, nr) :
    self.push(i, nl, nr)

    if r <= nl or nr <= l : return
    if l <= nl and nr <= r :
      self.lazy[i] += x
      self.push(i, nl, nr)
      return
  
    mid = (nl + nr) // 2
    self._add(l, r, x, i*2, nl, mid)
    self._add(l, r, x, i*2+1, mid, nr)
    self.L[i] = self.L[i*2] + self.L[i*2+1]

def sol() :
  N, M = map(int, input().split())
  lst = LazySegmentTree([0] * (N + 2))

  ans = []
  for _ in range(M) :
    q, *l = map(int, input().split())
    if q == 0 :
      s, e = l
      lst.add(s, e+1, 1)
    elif q == 1 :
      s, e = l
      ans.append(lst.query(s, e+1))
      print(lst.L)
  
  sys.stdout.write('\n'.join(map(str, ans)))

sol()

### 아이디어
- 켜진 스위치의 개수를 세야 하므로 누적합 세그트리를 만들어야 한다.
- update마다 변경량을 1씩 더해주는 것으로 구현하려 했는데, 리프노드를 제외한 곳의 처리를 어떻게 해야할지 감이 안온다. 잘못된 접근같다.